<a href="https://colab.research.google.com/github/romitbarua/MultiModalDeepFake/blob/main/UtilityNotebooks/GenerateMetadataFile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#mount the google drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [45]:
import os
import pandas as pd
import cv2
import dlib

In [43]:
SYM_PATH = '/content/drive/MyDrive/DeepFakeDetection'
%cd $SYM_PATH
%pip install -e .

/content/drive/MyDrive/DeepFakeDetection
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/MyDrive/DeepFakeDetection
  Running setup.py develop for DeepFake


In [57]:
from packages.DlibManager import DlibManager

In [32]:
main_directory = '/content/drive/MyDrive/DeepFakeDetection/data'
fake_av_celeb_metadata = pd.read_csv('/content/drive/MyDrive/DeepFakeDetection/data/FakeAVCeleb_v1.2/meta_data.csv')

In [77]:
check_dlib_validity = False

In [33]:
fake_av_celeb_metadata.rename(columns={'Unnamed: 9':'directory'}, inplace=True)

fake_av_celeb_metadata['directory'] = fake_av_celeb_metadata['directory'].str.replace('FakeAVCeleb', 'FakeAVCeleb_v1.2')
fake_av_celeb_metadata['full_path'] = main_directory + '/' +fake_av_celeb_metadata['directory'] + '/' + fake_av_celeb_metadata['path'] 
fake_av_celeb_metadata['video_source'] = 'FakeAVCeleb'

In [72]:
#get the phonemes and wavfiles
fake_av_phoneme_directory = 'phonetic_alignment_files'

phoneme_paths = []
wav_paths = []

for index, row in fake_av_celeb_metadata.iterrows():

    phoneme_path = row['full_path'].replace('FakeAVCeleb_v1.2', fake_av_phoneme_directory).replace('.mp4', '.TextGrid')
    if os.path.exists(phoneme_path):
      phoneme_paths.append(phoneme_path)
    else:
      phoneme_paths.append('')

    wav_path = row['full_path'].replace('.mp4', '.wav')
    if os.path.exists(wav_path):
      wav_paths.append(wav_path)
    else:
      wav_paths.append('')
    
fake_av_celeb_metadata['phoneme_path'] = phoneme_paths
fake_av_celeb_metadata['wav_path'] = wav_paths

In [74]:
def dlib_validity(video_paths):

  detector = dlib.get_frontal_face_detector()
  predictor = dlib.shape_predictor('/content/drive/MyDrive/DeepFakeDetection/model/shape_predictor_68_face_landmarks.dat')

  dlib_valid = []
  for idx, video_path in enumerate(video_paths):

    print(f'\r Video #{idx+1} out of {len(video_paths)}', end='')
    video = cv2.VideoCapture(video_path)

    try:
      dlib_video = DlibManager(detector=detector, predictor=predictor, video=video)
      dlib_valid.append(True)
    except:
      dlib_valid.append(False)

  return dlib_valid



In [69]:
if check_dlib_validity:
  fake_av_celeb_metadata['valid_dlib'] = dlib_validity(list(fake_av_celeb_metadata['full_path']))

Streaming output truncated to the last 5000 lines.
Video #16567 out of 21566
Video #16568 out of 21566
Video #16569 out of 21566
Video #16570 out of 21566
Video #16571 out of 21566
Video #16572 out of 21566
Video #16573 out of 21566
Video #16574 out of 21566
Video #16575 out of 21566
Video #16576 out of 21566
Video #16577 out of 21566
Video #16578 out of 21566
Video #16579 out of 21566
Video #16580 out of 21566
Video #16581 out of 21566
Video #16582 out of 21566
Video #16583 out of 21566
Video #16584 out of 21566
Video #16585 out of 21566
Video #16586 out of 21566
Video #16587 out of 21566
Video #16588 out of 21566
Video #16589 out of 21566
Video #16590 out of 21566
Video #16591 out of 21566
Video #16592 out of 21566
Video #16593 out of 21566
Video #16594 out of 21566
Video #16595 out of 21566
Video #16596 out of 21566
Video #16597 out of 21566
Video #16598 out of 21566
Video #16599 out of 21566
Video #16600 out of 21566
Video #16601 out of 21566
Video #16602 out of 21566
Video #16603 

In [75]:
fake_av_celeb_metadata.to_csv('/content/drive/MyDrive/DeepFakeDetection/data/FakeAVCeleb_v1.2/meta_data.csv_romit')

In [84]:
fake_av_celeb_metadata

,source,target1,target2,method,category,type,race,gender,path,directory,full_path,phoneme_path,valid_dlib,wav_path,video_source
0,id00076,-,-,real,A,RealVideo-RealAudio,African,men,00109.mp4,FakeAVCeleb_v1.2/RealVideo-RealAudio/African/m...,/content/drive/MyDrive/DeepFakeDetection/data/...,/content/drive/MyDrive/DeepFakeDetection/data/...,True,/content/drive/MyDrive/DeepFakeDetection/data/...,FakeAVCeleb
1,id00166,-,-,real,A,RealVideo-RealAudio,African,men,00010.mp4,FakeAVCeleb_v1.2/RealVideo-RealAudio/African/m...,/content/drive/MyDrive/DeepFakeDetection/data/...,/content/drive/MyDrive/DeepFakeDetection/data/...,True,/content/drive/MyDrive/DeepFakeDetection/data/...,FakeAVCeleb
2,id00173,-,-,real,A,RealVideo-RealAudio,African,men,00118.mp4,FakeAVCeleb_v1.2/RealVideo-RealAudio/African/m...,/content/drive/MyDrive/DeepFakeDetection/data/...,/content/drive/MyDrive/DeepFakeDetection/data/...,True,/content/drive/MyDrive/DeepFakeDetection/data/...,FakeAVCeleb
3,id00366,-,-,real,A,RealVideo-RealAudio,African,men,00118.mp4,FakeAVCeleb_v1.2/RealVideo-RealAudio/African/m...,/content/drive/MyDrive/DeepFakeDetection/data/...,/content/drive/MyDrive/DeepFakeDetection/data/...,False,/content/drive/MyDrive/DeepFakeDetection/data/...,FakeAVCeleb
4,id00391,-,-,real,A,RealVideo-RealAudio,African,men,00052.mp4,FakeAVCeleb_v1.2/RealVideo-RealAudio/African/m...,/content/drive/MyDrive/DeepFakeDetection/data/...,/content/drive/MyDrive/DeepFakeDetection/data/...,True,/content/drive/MyDrive/DeepFakeDetection/data/...,FakeAVCeleb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21561,id07689,id06254,-,wav2lip,D,FakeVideo-FakeAudio,Asian (South),women,00028_id06254_wavtolip.mp4,FakeAVCeleb_v1.2/FakeVideo-FakeAudio/Asian (So...,/content/drive/MyDrive/DeepFakeDetection/data/...,,True,/content/drive/MyDrive/DeepFakeDetection/data/...,FakeAVCeleb
21562,id07689,id06343,-,wav2lip,D,FakeVideo-FakeAudio,Asian (South),women,00028_id06343_wavtolip.mp4,FakeAVCeleb_v1.2/FakeVideo-FakeAudio/Asian (So...,/content/drive/MyDrive/DeepFakeDetection/data/...,,True,/content/drive/MyDrive/DeepFakeDetection/data/...,FakeAVCeleb
21563,id07689,id07008,-,wav2lip,D,FakeVideo-FakeAudio,Asian (South),women,00028_id07008_wavtolip.mp4,FakeAVCeleb_v1.2/FakeVideo-FakeAudio/Asian (So...,/content/drive/MyDrive/DeepFakeDetection/data/...,,True,/content/drive/MyDrive/DeepFakeDetection/data/...,FakeAVCeleb
21564,id07689,id07377,-,wav2lip,D,FakeVideo-FakeAudio,Asian (South),women,00028_id07377_wavtolip.mp4,FakeAVCeleb_v1.2/FakeVideo-FakeAudio/Asian (So...,/content/drive/MyDrive/DeepFakeDetection/data/...,,True,/content/drive/MyDrive/DeepFakeDetection/data/...,FakeAVCeleb


In [79]:
#Generate VoxCeleb Metadata file
vox_celeb_directory = '/content/drive/MyDrive/DeepFakeDetection/data/VoxCeleb'

In [90]:
ids = list(os.walk(vox_celeb_directory))[0][1]

sources = []
paths = []
directories = []
full_paths = []
wav_paths = []

for id in ids:
  id_directory = f'{vox_celeb_directory}/{id}'
  for walk_path in os.walk(id_directory):
    files = walk_path[2]
    for file_name in files:
      if '.mp4' in file_name:
        sources.append(id)
        paths.append(file_name)
        directories.append(id_directory)

        full_path = os.path.join(id_directory, file_name)
        full_paths.append(full_path)

        wav_path = full_path.replace('.mp4', '.wav')
        if os.path.exists(wav_path):
          wav_paths.append(wav_path)
        else:
          wav_paths.append('')

vox_celeb_metadata = pd.DataFrame({'source':sources, 'path':paths, 'directory':directories, 'full_path':full_paths, 'wav_path':wav_paths})

In [103]:
#add once we have phoneme structure
vox_celeb_metadata['phoneme_path'] = ''

In [93]:
if check_dlib_validity:
  vox_celeb_metadata['valid_dlib'] = dlib_validity(list(vox_celeb_metadata['full_path']))



 Video #1 out of 1394
 Video #2 out of 1394
 Video #3 out of 1394
 Video #4 out of 1394
 Video #5 out of 1394
 Video #6 out of 1394
 Video #7 out of 1394
 Video #8 out of 1394
 Video #9 out of 1394
 Video #10 out of 1394
 Video #11 out of 1394
 Video #12 out of 1394
 Video #13 out of 1394
 Video #14 out of 1394
 Video #15 out of 1394
 Video #16 out of 1394
 Video #17 out of 1394
 Video #18 out of 1394
 Video #19 out of 1394
 Video #20 out of 1394
 Video #21 out of 1394
 Video #22 out of 1394
 Video #23 out of 1394
 Video #24 out of 1394
 Video #25 out of 1394
 Video #26 out of 1394
 Video #27 out of 1394
 Video #28 out of 1394
 Video #29 out of 1394
 Video #30 out of 1394
 Video #31 out of 1394
 Video #32 out of 1394
 Video #33 out of 1394
 Video #34 out of 1394
 Video #35 out of 1394
 Video #36 out of 1394
 Video #37 out of 1394
 Video #38 out of 1394
 Video #39 out of 1394
 Video #40 out of 1394
 Video #41 out of 1394
 Video #42 out of 1394
 Video #43 out of 1394
 Video #44 out of 13

In [100]:
vox_celeb_metadata['type'] = 'RealVideo-RealAudio'
vox_celeb_metadata['method'] = 'real'
vox_celeb_metadata['video_source'] = 'VoxCeleb'


In [101]:
full_data = pd.concat((vox_celeb_metadata, fake_av_celeb_metadata))
full_data.columns

Index(['source', 'path', 'directory', 'full_path', 'wav_path', 'valid_dlib',
       'type', 'method', 'video_source', 'target1', 'target2', 'category',
       'race', 'gender', 'phoneme_path'],
      dtype='object')

In [105]:
full_data.to_csv('/content/drive/MyDrive/DeepFakeDetection/data/full_metadata.csv')